# Data Pre Processing

Making sure the afttables dataset is cleaned and prepped for analysis

Import required libraries

In [330]:
import pandas as pd
import matplotlib.pyplot as plt

Import the dataset

In [331]:
matches = pd.read_csv('../afltables_matches.csv', index_col = 0)
matches.head()

,yearly_match_number,year,round,day,date,time,attendance,venue,home_team,home_q1_goals,...,away_q2_points,away_q2_score,away_q3_goals,away_q3_points,away_q3_score,away_q4_goals,away_q4_points,away_q4_score,away_final_score,is_bye
0,0,1897,1,Sat,08-May-1897,3:00 PM,"3,000",Brunswick St,Fitzroy,0.0,...,3.0,9.0,2.0,3.0,15.0,2.0,4.0,16.0,16.0,0
1,1,1897,1,Sat,08-May-1897,3:00 PM,NaN,Victoria Park,Collingwood,1.0,...,2.0,14.0,2.0,4.0,16.0,2.0,4.0,16.0,16.0,0
2,2,1897,1,Sat,08-May-1897,3:00 PM,NaN,Corio Oval,Geelong,0.0,...,4.0,22.0,6.0,4.0,40.0,7.0,5.0,47.0,47.0,0
3,3,1897,1,Sat,08-May-1897,3:00 PM,NaN,Lake Oval,South Melbourne,0.0,...,3.0,27.0,5.0,4.0,34.0,6.0,8.0,44.0,44.0,0
4,4,1897,2,Sat,15-May-1897,3:00 PM,NaN,Lake Oval,South Melbourne,2.0,...,2.0,14.0,2.0,3.0,15.0,5.0,6.0,36.0,36.0,0


# Drop Redundant Data Types
Drop the initial redundant features.

- `year` : The matches' date will cover this feature
- `day` : When converted to a datetime this will be easily found with the date feature
- All Q scores : For the ELO Ratings the quarter time scores are not required

In [332]:
home_q_scores = ['home_q1_goals', 'home_q1_points', 'home_q1_score',
                 'home_q2_goals', 'home_q2_points', 'home_q2_score',
                 'home_q3_goals', 'home_q3_points', 'home_q3_score',
                 'home_q4_goals', 'home_q4_points', 'home_q4_score']
away_q_scores = ['away_q1_goals', 'away_q1_points', 'away_q1_score',
                 'away_q2_goals', 'away_q2_points', 'away_q2_score',
                 'away_q3_goals', 'away_q3_points', 'away_q3_score',
                 'away_q4_goals', 'away_q4_points', 'away_q4_score']
matches.drop(['year', 'day'] + home_q_scores + away_q_scores, axis = 1, inplace=True)

# Fix Data Types

In [333]:
matches.dtypes

yearly_match_number      int64
round                   object
date                    object
time                    object
attendance              object
venue                   object
home_team               object
home_final_score       float64
away_team               object
away_final_score       float64
is_bye                   int64
dtype: object

## Time / Date Features

In [334]:
matches.date = pd.to_datetime(matches.date)
matches.time = pd.to_datetime(matches.time).dt.time

## Categorical Features

In [335]:
matches.yearly_match_number = pd.Categorical(matches.yearly_match_number, ordered=True)
matches['round'] = pd.Categorical(matches['round'], ordered=True, categories=['1', '2', '3', '4', '5', '6', '7', '8',
                                                           '9', '10', '11', '12', '13', '14', '15', '16',
                                                           '17', '18', '19', '20', '21', '22', '23',
                                                           '24', 'Elimination Final', 'Qualifying Final',
                                                           'Semi Final', 'Preliminary Final', 'Grand Final'])
matches.venue = pd.Categorical(matches.venue)
matches.home_team = pd.Categorical(matches.home_team)
matches.away_team = pd.Categorical(matches.away_team)

## Boolean Features

In [336]:
matches.is_bye = matches.is_bye.astype(bool)

# Missing Values

In [337]:
matches.isnull().sum()

yearly_match_number       0
round                     0
date                    469
time                    469
attendance             3559
venue                   469
home_team                 0
home_final_score        469
away_team               469
away_final_score        469
is_bye                    0
dtype: int64

## Remove Byes
As byes don't affect the ELO rating it's not worth having them in the dataset. With all those rows removed we can also drop the column

In [338]:
matches = matches[~matches['is_bye']].drop('is_bye', axis=1).reset_index()

## Drop Attendance
As a significant number of values are missing we will simply drop the feature from the dataset. It's also not really relevant to calculating the ELO rating either

In [339]:
matches.drop('attendance', axis=1, inplace=True)

# Outliers

## Home & Away Final Scores

We are unlikely to change anything here as some results are actually outliers, however, we want to make sure the maximum and minimum results are true results and not a mistake in the recording process.

Actual Home Max: Fitzroy with 238
Actual Away Max: Geelong with 239
Actual Home Min: St Kilda with 3
Actual Away Min: St Kilda with 1

### Home Scores

In [340]:
home_stat_summary = matches.home_final_score.describe()
home_iqr = home_stat_summary['75%'] - home_stat_summary['25%']
home_lower_fence = home_stat_summary['25%'] - (home_iqr * 1.5)
home_upper_fence = home_stat_summary['75%'] + (home_iqr * 1.5)
home_outlier_mask = (matches.home_final_score < home_lower_fence) | (matches.home_final_score > home_upper_fence)
home_outliers = matches[home_outlier_mask].home_final_score
home_max = matches.iloc[home_outliers.idxmax()]
home_min = matches.iloc[home_outliers.idxmin()]
print(f"Dataset home max {home_max.home_team} with {home_max.home_final_score}")
print(f"Dataset home min {home_min.home_team} with {home_min.home_final_score}")

Dataset home max Fitzroy with 238.0
Dataset home min St Kilda with 3.0


In [341]:
away_stat_summary = matches.away_final_score.describe()
away_iqr = away_stat_summary['75%'] - away_stat_summary['25%']
away_lower_fence = away_stat_summary['25%'] - (away_iqr * 1.5)
away_upper_fence = away_stat_summary['75%'] + (away_iqr * 1.5)
away_outlier_mask = (matches.away_final_score < away_lower_fence) | (matches.away_final_score > away_upper_fence)
away_outliers = matches[away_outlier_mask].away_final_score
away_max = matches.iloc[away_outliers.idxmax()]
away_min = matches.iloc[away_outliers.idxmin()]
print(f"Dataset away max {away_max.away_team} with {away_max.away_final_score}")
print(f"Dataset away min {away_min.away_team} with {away_min.away_final_score}")

Dataset away max Geelong with 239.0
Dataset away min St Kilda with 161.0


Interestingly enough the minimum away value was not an outlier :). The maximum and minimum of both were the actual values suggesting there were not way incorrect values in the dateset

# Check Categorical Data

## Home & Away Team Names
Make sure the names are consistent throughout the dataset

### Home Team

In [342]:
matches.home_team.value_counts()

Collingwood               1282
Carlton                   1268
Essendon                  1248
Geelong                   1245
Melbourne                 1212
St Kilda                  1194
Richmond                  1148
Hawthorn                  1010
Fitzroy                    951
North Melbourne            892
South Melbourne            782
Footscray                  712
Sydney                     450
West Coast                 402
Adelaide                   341
Fremantle                  292
Brisbane Lions             278
Port Adelaide              276
Western Bulldogs           270
Brisbane Bears             111
Gold Coast                 109
Kangaroos                  105
Greater Western Sydney      99
University                  63
Name: home_team, dtype: int64

Looks like North Melbourne were called the `Kangaroos` sometimes for the home team in the dataset

In [343]:
matches.loc[matches.home_team == "Kangaroos", 'home_team'] = "North Melbourne"

In [344]:
matches.away_team.value_counts()

Collingwood               1285
Carlton                   1253
Essendon                  1239
Geelong                   1221
Melbourne                 1217
St Kilda                  1210
Richmond                  1153
Hawthorn                  1005
Fitzroy                    968
North Melbourne            902
South Melbourne            779
Footscray                  719
Sydney                     449
West Coast                 394
Adelaide                   347
Fremantle                  290
Western Bulldogs           278
Port Adelaide              272
Brisbane Lions             270
Brisbane Bears             111
Gold Coast                 106
Greater Western Sydney     105
Kangaroos                  104
University                  63
Name: away_team, dtype: int64

Same as the home team it appears North Melbourne were labelled as the `Kangaroos` in the away team column as well. 

In [345]:
matches.loc[matches.away_team == "Kangaroos", 'away_team'] = "North Melbourne"

### Fix Home & Away Team Name Data Types

Because we removed `Kangaroos` as a value we need to fix up the categories of the data types

In [346]:
matches.away_team = matches.away_team.cat.remove_unused_categories()
matches.home_team = matches.home_team.cat.remove_unused_categories()

# Save Data
Will save the data in parquet format to ensure the data types remain for the Elo Ratings

In [329]:
# Drop old index value
matches.drop('index', axis=1, inplace=True)
matches.to_pickle('../data/aflmatches_wo_elo.pypickle')